In [6]:
!pip install spacy

use spaCy, a  Python library for natural language processing, both to tokenize text (i.e., turn text into a list of words) and for its database of word vectors

In [7]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 852.3MB 1.2MB/s 
  Stored in directory: /tmp/pip-ephem-wheel-cache-vjgtvxud/wheels/0d/bc/67/e6a9108ab86cd076703af19ad4e0f02f57381ac6583df16249
Successfully built en-core-web-lg

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_lg -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en_core_web_lg

    You can now load the model via spacy.load('en_core_web_lg')



In [0]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [9]:
nlp.vocab['namaste'].vector # replace cheese with whatever word you want!

array([ 1.6712e-01, -1.2405e-01, -6.6796e-01,  4.7600e-01,  5.9755e-01,
        5.1452e-01, -2.3764e-01, -8.0116e-02, -3.6755e-02, -8.4024e-01,
        5.6909e-02, -3.1647e-02,  2.7034e-01, -5.1196e-01, -4.5864e-01,
        1.3465e-01,  1.0157e-01, -6.7575e-01, -4.3799e-02, -1.2819e-01,
       -2.3192e-01,  3.6284e-01,  1.9340e-01,  5.0953e-01,  5.9551e-02,
       -7.5207e-01, -2.3583e-02,  1.6140e-01, -5.2897e-02,  1.0111e+00,
        2.3660e-01,  1.7796e-01, -3.8463e-01,  6.7698e-02, -1.3832e-01,
        2.6871e-01, -8.1885e-02, -1.1268e-01, -3.4461e-01, -5.8820e-01,
       -2.1476e-02, -2.1207e-01,  4.0075e-01, -3.1574e-01,  5.8029e-01,
       -1.2500e-01,  8.0864e-02,  3.2958e-01, -1.4880e-01,  1.1263e-01,
        2.8775e-01,  1.9306e-01,  5.8157e-02, -5.7205e-01,  3.3520e-01,
        2.5792e-01,  4.6802e-01,  6.6447e-01, -1.8832e-01, -5.3755e-01,
       -5.3893e-01, -5.7998e-01,  1.1152e-01, -2.6772e-01, -2.0913e-01,
       -9.9323e-01, -7.1427e-01,  9.4896e-01, -5.1612e-01, -1.36

In [10]:
!curl -L -O http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9684k  100 9684k    0     0  1925k      0  0:00:05  0:00:05 --:--:-- 2278k


In [11]:
!unzip cornell_movie_dialogs_corpus.zip

Archive:  cornell_movie_dialogs_corpus.zip
   creating: cornell movie-dialogs corpus/
  inflating: cornell movie-dialogs corpus/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/cornell movie-dialogs corpus/
  inflating: __MACOSX/cornell movie-dialogs corpus/._.DS_Store  
  inflating: cornell movie-dialogs corpus/chameleons.pdf  
  inflating: __MACOSX/cornell movie-dialogs corpus/._chameleons.pdf  
  inflating: cornell movie-dialogs corpus/movie_characters_metadata.txt  
  inflating: cornell movie-dialogs corpus/movie_conversations.txt  
  inflating: cornell movie-dialogs corpus/movie_lines.txt  
  inflating: cornell movie-dialogs corpus/movie_titles_metadata.txt  
  inflating: cornell movie-dialogs corpus/raw_script_urls.txt  
  inflating: cornell movie-dialogs corpus/README.txt  
  inflating: __MACOSX/cornell movie-dialogs corpus/._README.txt  


In [0]:
movie_lines = {}
for line in open("./cornell movie-dialogs corpus/movie_lines.txt",
                 encoding="latin1"):
    line = line.strip()
    parts = line.split(" +++$+++ ")
    if len(parts) == 5:
        movie_lines[parts[0]] = parts[4]
    else:
        movie_lines[parts[0]] = ""

In [0]:
import json
responses = {}
for line in open("./cornell movie-dialogs corpus/movie_conversations.txt",
                 encoding="latin1"):
    line = line.strip()
    parts = line.split(" +++$+++ ")
    line_ids = json.loads(parts[3].replace("'", '"'))
    for first, second in zip(line_ids[:-1], line_ids[1:]):
        responses[first] = second

In [14]:
import random
for pair in random.sample(responses.items(), 5):
    print("A:", movie_lines[pair[0]])
    print("B:", movie_lines[pair[1]])
    print()

A: Well -- nice to know life exists outside the office.
B: Yes.

A: I'm sorry if I caught you at a bad time.
B: We're just a little tense right now...  It's okay.

A: I really don't have much of an appetite.
B: Just wait.

A: I want my man back.
B: Just a moment, how did you get in here?

A: I do, she's real funny.
B: He likes her better than me.



In [15]:
import numpy as np
def sentence_mean(nlp, s):
    if s == "":
        s = " "
    doc = nlp(s, disable=['tagger', 'parser'])
    return np.mean(np.array([w.vector for w in doc]), axis=0)
sentence_mean(nlp, "This... is a test.").shape

(300,)

In [16]:
!pip install simpleneighbors

     |████████████████████████████████| 645kB 7.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/d4/78/3a/1c913a402f4d84f68eb0ea4f7c0de0d392725415c8ebe51068
Successfully built annoy


In [17]:
from simpleneighbors import SimpleNeighbors

nns = SimpleNeighbors(300)
for i, line_id in enumerate(random.sample(list(responses.keys()), 10000)):
    # show progress
    if i % 1000 == 0: print(i, line_id, movie_lines[line_id])
    line_text = movie_lines[line_id]
    summary_vector = sentence_mean(nlp, line_text)
    if np.any(summary_vector):
        nns.add_one(line_id, summary_vector)
nns.build()

0 L151893 Maybe you should go out with him. If you keep avoiding him he'll get suspicious. Besides, what better way to not get busted than to date the cop who's investigating you.
1000 L10285 I saw something!  I'm not going to go back there and say I didn't see it when I did.  I'm sorry.
2000 L552232 Well, what about the Positive Control, the safety catch?  Don't the planes automatically come back unless they get a second order?
3000 L104104 My walkie talkie's busted. I can't tell what's going on.
4000 L162670 Yeah, right. She wishes. Look at that big ass.
5000 L422966 I'm a difficult but brilliant junior officer who in about twenty minutes will save the world from nuclear disaster.
6000 L482346 This has nothing to do with him.
7000 L189589 You want my number?
8000 L100886 Yeah right, and which ever of us is still alive at the end of the week inherits all Vincent Price's cash.
9000 L356235 Sit down a minute.


In [18]:
sentence = "I like making bots."
picked = nns.nearest(sentence_mean(nlp, sentence), 5)[0]
response_line_id = responses[picked]

print("Your line:\n\t", sentence)
print("Most similar turn:\n\t", movie_lines[picked])
print("Response to most similar turn:\n\t", movie_lines[response_line_id])

Your line:
	 I like making bots.
Most similar turn:
	 I like you.
Response to most similar turn:
	 Do you?


In [19]:
!pip install https://github.com/aparrish/semanticsimilaritychatbot/archive/master.zip

     | 153kB 1.6MB/s
  Stored in directory: /tmp/pip-ephem-wheel-cache-jjyll47y/wheels/f7/af/8e/8a8fbef31bfbfc3b935425efa03db03825795d85f4e23f8255
Successfully built semanticsimilaritychatbot


In [0]:
from semanticsimilaritychatbot import SemanticSimilarityChatbot
chatbot = SemanticSimilarityChatbot(nlp, 300)

In [0]:
sample_n = 10000
for first_id, second_id in random.sample(list(responses.items()), sample_n):
    chatbot.add_pair(movie_lines[first_id], movie_lines[second_id])
chatbot.build()

In [22]:
print(chatbot.response_for("Hello computer!"))

Show's over unless I get to see what you've been working on after hours.


In [23]:
my_turn = "The weather's nice today, don't you think?"
for i in range(5, 51, 5):
    print("picking from", i, "possible responses:")
    print(chatbot.response_for(my_turn, i))
    print()

picking from 5 possible responses:
Dan, are you really that paranoid?  Let's face it, Eddie Blake was no Mother Teresa.

picking from 10 possible responses:
Don't bother.

picking from 15 possible responses:
Dan, are you really that paranoid?  Let's face it, Eddie Blake was no Mother Teresa.

picking from 20 possible responses:
What?

picking from 25 possible responses:
What'll it do if they pull the plug?

picking from 30 possible responses:
Don't bother.

picking from 35 possible responses:
Speak.

picking from 40 possible responses:
No, I had a dream about daddy.  Do you think I'll ever meet anyone like him?

picking from 45 possible responses:
It's not that easy.

picking from 50 possible responses:
Jeez, Mom, stop trippin'.



In [0]:
chatbot.save("movielines-10k-sample")

In [0]:
chatbot = SemanticSimilarityChatbot.load("movielines-10k-sample", nlp)

In [26]:
print(chatbot.response_for("I'm going to eat something."))

Oh. Well, wasn't it Shakespeare who said: "When strangers do meet they should erelong see one another again"?


In [0]:
from google.colab import files
files.download('movielines-10k-sample.annoy')
files.download('movielines-10k-sample-data.pkl')
files.download('movielines-10k-sample-chatbot.pkl')

In [0]:
chatbot_html = """
<style type="text/css">#log p { margin: 5px; font-family: sans-serif; }</style>
<div id="log"
     style="box-sizing: border-box;
            width: 600px;
            height: 32em;
            border: 1px grey solid;
            padding: 2px;
            overflow: scroll;">
</div>
<input type="text" id="typehere" placeholder="type here!"
       style="box-sizing: border-box;
              width: 600px;
              margin-top: 5px;">
<script>
function paraWithText(t) {
    let tn = document.createTextNode(t);
    let ptag = document.createElement('p');
    ptag.appendChild(tn);
    return ptag;
}
document.querySelector('#typehere').onchange = async function() {
    let inputField = document.querySelector('#typehere');
    let val = inputField.value;
    inputField.value = "";
    let resp = await getResp(val);
    let objDiv = document.getElementById("log");
    objDiv.appendChild(paraWithText('😀: ' + val));
    objDiv.appendChild(paraWithText('🤖: ' + resp));
    objDiv.scrollTop = objDiv.scrollHeight;
};
async function colabGetResp(val) {
    let resp = await google.colab.kernel.invokeFunction(
        'notebook.get_response', [val], {});
    return resp.data['application/json']['result'];
}
async function webGetResp(val) {
    let resp = await fetch("/response.json?sentence=" + 
        encodeURIComponent(val));
    let data = await resp.json();
    return data['result'];
}
</script>
"""

In [31]:
import IPython
from google.colab import output

display(IPython.display.HTML(chatbot_html + \
                             "<script>let getResp = colabGetResp;</script>"))

def get_response(val):
    resp = chatbot.response_for(val)
    return IPython.display.JSON({'result': resp})

output.register_callback('notebook.get_response', get_response)